In [ ]:
#Load Dependencies
import time
import json
import os
import random
from bs4 import BeautifulSoup
import requests

### What's currently missing code-wise:

Removed proxy implementation code for the moment
Sub-cat and part code does not have implementation for checking already scraped code (should be able to just compare url's present in sub-cat or parts to see if that part has been scraped or not)
Search feature to only scrape parts by specific priority manufacturers



In [ ]:
#Manufacturer scraping code(complete)

url = "https://www.rockauto.com/en/catalog/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser').find_all('div', attrs={'class', 'ranavnode'})
soup
soup_filter = []
makes_list = []
# Find US Market Only
for x in soup:
    soup_filter.append( x.find('a', attrs={'class', 'navlabellink'}) )

# Get [Make, Year, Model, Link]
for x in soup_filter:
    makes_list.append( {'make': x.get_text(), 'link': 'https://www.rockauto.com' + str( x.get('href') ) })
makes_list

In [ ]:
#Makelist by Manufacturer (complete)

makes_list = makes_list[:]
makes = []
for car in makes_list:
    search_make = car["make"]
    url = car["link"]
    
    years_list = []

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser').find_all('div', attrs={'class': 'ranavnode'})
    soup = soup[1:]
    soup_filter = []

    # Find US Market Only
    for x in soup:
        if 'US' in next(x.children)['value']:
            soup_filter.append( x.find('a', attrs={'class': 'navlabellink'}) )
    # Get [Make, Year, Model, Link]
    for x in soup_filter:
        makes.append({'make': search_make, 'year': x.get_text(), 'link': 'https://www.rockauto.com' + str( x.get('href') ) })
    
makes

In [ ]:
#Filter to vehicles after 2015
makes = makes[makes['year'] > 2015]

In [ ]:
#Vehicle model list (complete)

models = []
output_file = "scraped_models.json"

for index, (search_make, url, search_year) in enumerate(zip(makes["make"], makes["link"], makes["year"]), 1):

    response = requests.get(url)
    time.sleep(random.uniform(1, 3))  # Introduce a random delay between 1 and 3 seconds
    soup = BeautifulSoup(response.text, 'html.parser').find_all('div', attrs={'class': 'ranavnode'})
    soup = soup[1:]
    soup_filter = []

    for x in soup:
        if 'US' in next(x.children)['value']:
            soup_filter.append(x.find('a', attrs={'class', 'navlabellink'}))

    # Get [Make, Year, Model, Link]
    for x in soup_filter:
        model = {'make': search_make, 'year': search_year, 'model': x.get_text(), 'link': 'https://www.rockauto.com' + str(x.get('href'))}
        models.append(model)
        with open(output_file, 'a') as f:
            json.dump(model, f, indent=4)
            f.write('\n')
        
        time.sleep(random.uniform(1, 3))  # Introduce a random delay between 1 and 3 seconds before next request

# Print response after all iterations are done
print("Response:", response.text)


In [ ]:
#Engine scraping code (complete)
models = models[:]
output_file = "scraped_engines.json"

for car in models:
    search_make = car["make"]
    url = car["link"]
    search_year = car["year"]
    search_model = car["model"]

    # Check if this car already exists in engines
    if any(engine['make'] == search_make and engine['year'] == search_year and engine['model'] == search_model for engine in engines):
        continue  # Skip this car if it already exists

    response = requests.get(url)
    time.sleep(random.uniform(1, 3))  # Introduce a random delay between 1 and 3 seconds

    soup = BeautifulSoup(response.text, 'html.parser').find_all('div', attrs={'class': 'ranavnode'})
    soup = soup[1:]
    soup_filter = []

    for x in soup:
        if 'US' in next(x.children)['value']:
            soup_filter.append( x.find('a', attrs={'class', 'navlabellink'}) )

    # Get [Make, Year, Model, Link]
    for x in soup_filter:
        
        engine = {'make': search_make, 'year': search_year, 'model': search_model, 'engine': x.get_text(), 'link': 'https://www.rockauto.com' + str( x.get('href') ) }
        engines.append(engine)
        with open(output_file, 'a') as f:
            json.dump(engine, f, indent=4)
            f.write('\n')
        
        time.sleep(random.uniform(1, 3))  # Introduce a random delay between 1 and 3 seconds before next request
    
engines


In [ ]:
#categories scrape code (with rotating user agents, vastly incomplete as well due to lack of proxies)
from fake_useragent import UserAgent
import random 
engines = engines[0:1]
categories = []
e = engines[:]
output_file = "categories.json"
ua = UserAgent()

for car in e:

    search_make = car["make"]
    url = car["link"]
    search_year = car["year"]
    search_model = car["model"]
    search_engine = car["engine"]
    headers = {'User-Agent': ua.random}
    response = requests.get(url, headers = headers)
    if any(engine['make'] == search_make and engine['year'] == search_year and engine['model'] == search_model and engine['engine'] == search_engine for engine in categories):
        continue  # Skip this car if it already exists
  
    soup = BeautifulSoup(response.text, 'html.parser').find_all('a', attrs={'class', 'navlabellink'})[4:]
    for x in soup:
        response = requests.get('https://www.rockauto.com' + str( x.get('href') ))
        
        soup = BeautifulSoup(response.text, 'html.parser').find_all('a', attrs={'class', 'navlabellink'})[5:]
        categories.append({'make': search_make, 'year': search_year, 'model': search_model, 'engine': search_engine, 'category': x.get_text()})
        print(categories)
        with open(output_file, 'a') as f:
                        json.dump(categories[-1], f, indent=4)
                        f.write('\n')


In [ ]:
#Sub-cat scrape code (incomplete, categories isn't done scraping)
categories = categories[0:2]
subcat = []
for car in categories:
    search_make = car["make"]
    url = car["link"]
    search_year = car["year"]
    search_model = car["model"]
    search_engine = car["engine"]
    search_category = car["category"]
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser').find_all('a', attrs={'class', 'navlabellink'})[5:]

    for x in soup:
        subcat.append( {'make': search_make, 'year': search_year, 'model': search_model, 'engine': search_engine, 'category': search_category, 'sub_category': x.get_text(), 'link': 'https://www.rockauto.com' + str( x.get('href') ) })
        
    
subcat

In [ ]:
#part_list scrape code (incomplete, categories and subcategories aren't done scraping)

part_list = []
for car in subcat:
    search_make = car["make"]
    url = car["link"]
    search_year = car["year"]
    search_model = car["model"]
    search_engine = car["engine"]
    search_category = car["category"]
    search_subcat = car["sub_category"]
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')


    for x in soup:    
        #Parts
        for product_block in soup.find_all('div', class_='listing-text-row-moreinfo-truck'):  
            # Create a dictionary for this product
            product_dict = {
                'make': search_make, 'year': search_year, 'model': search_model, 'engine': search_engine, 'category': search_category, 'sub_category': search_subcat,
                'manufacturer': product_block.find('span', class_='listing-final-manufacturer').text,
                'part_number': product_block.find('span', class_='listing-final-partnumber').text
                #'description': product_block.find('span', class_='span-link-underline-remover').text,
            }
            try:
                product_dict['link'] = product_block.find('a', class_='ra-btn ra-btn-moreinfo')['href']
            except TypeError:
                product_dict['link'] = None

            
            part_list.append(product_dict)
        
    
part_list

In [ ]:
#Search make Query
search_query = {'make': "FORD", 'model': "ESCAPE", 'year': "2023"}

In [ ]:
#Code to scrape part_list (works completely, however unable to run without proxies)
#Has to be run in one shot, unable to do partial scrapes and pick up on where the last iteration left off
engines = engines[:]
categories = []
part_list = []
output_file = "part_list.json"

# Initialize an empty filter
filt = {}

# Add values to the filter if they exist in search_query
for key, value in search_query.items():
    if value: 
        filt[key] = value
for car in engines:
    search_make = car["make"]
    url = car["link"]
    search_year = car["year"]
    search_model = car["model"]
    search_engine = car["engine"]
    c = {
    "make": search_make,
    "link": url,
    "year": search_year,
    "model": search_model,
    "engine": search_engine
    }
    all_values_match = True
    for key in filt:
        if key in c and filt[key] != c[key]:
            all_values_match = False
            break

    if all_values_match:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser').find_all('a', attrs={'class', 'navlabellink'})[4:]
        time.sleep(random.uniform(2, 3))
        for x in soup:
            #categories
            response = requests.get('https://www.rockauto.com' + str( x.get('href') ))

            soup = BeautifulSoup(response.text, 'html.parser').find_all('a', attrs={'class', 'navlabellink'})[5:]
            time.sleep(random.uniform(2, 3))
            for y in soup:
                #Sub-cat
                response = requests.get('https://www.rockauto.com' + str( y.get('href') ))
                soup = BeautifulSoup(response.text, 'html.parser')
                time.sleep(random.uniform(2, 3))
                for j in soup:
                    #Parts
                    for product_block in soup.find_all('div', class_='listing-text-row-moreinfo-truck'):  
                        # Create a dictionary for this product
                        product_dict = {
                            'make': search_make, 'year': search_year, 'model': search_model, 'engine': search_engine, 'category': x.get_text(), 'sub_category': y.get_text(),
                            'manufacturer': product_block.find('span', class_='listing-final-manufacturer').text,
                            'part_number': product_block.find('span', class_='listing-final-partnumber').text
                            #'description': product_block.find('span', class_='span-link-underline-remover').text,
                        }
                        try:
                            product_dict['link'] = product_block.find('a', class_='ra-btn ra-btn-moreinfo')['href']
                        except TypeError:
                            product_dict['link'] = None

                        with open(output_file, 'a') as f:
                            json.dump(product_dict, f, indent=4)
                            f.write('\n')
                        part_list.append(product_dict)
                        time.sleep(random.uniform(2, 3))
part_list

In [ ]:
#Load scraped models (complete)
data_list = []

with open('scraped_models.txt', 'r') as file:
    data = file.read().split('\n}\n{')
    for item in data:
        item = item.replace('{', '').replace('}', '').split(',\n')
        dict_data = {}
        for element in item:
            key, value = element.strip().split(': ')
            dict_data[key.strip('"')] = value.strip('"')
        data_list.append(dict_data)

data_list
models = data_list

In [ ]:
#Load current scraped engines (complete)

data_list = []

with open('scraped_engines.json', 'r') as file:
    data = file.read().split('\n}\n{')
    for item in data:
        item = item.replace('{', '').replace('}', '').split(',\n')
        dict_data = {}
        for element in item:
            key, value = element.strip().split(': ')
            dict_data[key.strip('"')] = value.strip('"')
        data_list.append(dict_data)

data_list
engines = data_list
engines

In [ ]:
#Load current scraped categories (don't load this, scrape from scratch)(lacks URL link, so start scraping from scratch)
data_list = []

with open('categories.json', 'r') as file:
    data = file.read().split('\n}\n{')
    for item in data:
        item = item.replace('{', '').replace('}', '').split(',\n')
        dict_data = {}
        for element in item:
            key, value = element.strip().split(': ')
            dict_data[key.strip('"')] = value.strip('"')
        data_list.append(dict_data)

data_list
categories = data_list

In [ ]:
#Load current scraped part_list (very incomplete, just here to look at what the final product looks like)

data_list = []

with open('part_list.json', 'r') as file:
    data = file.read().split('\n}\n{')
    for item in data:
        item = item.replace('{', '').replace('}', '').split(',\n')
        dict_data = {}
        for element in item:
            key, value = element.strip().split(': ')
            dict_data[key.strip('"')] = value.strip('"')
        data_list.append(dict_data)

data_list
part_list = data_list